In [ ]:
!git clone -b train --single-branch https://github.com/MLOpshub/extractive-qa-mlops.git
%cd extractive-qa-mlops

!pip -q install uv
!uv sync


In [ ]:
from google.colab import drive
drive.mount("/content/drive")


In [ ]:
!cd /content/extractive-qa-mlops
!pip -q install -e .

In [ ]:
import os

os.environ["MODELS_DIR"] = "/content/drive/MyDrive/extractive-qa-mlops/artifacts/models"
os.environ["MLRUNS_DIR"] = "/content/drive/MyDrive/extractive-qa-mlops/mlruns"


Running sweep which includes the baseline traing

In [ ]:
!uv run python scripts/sweep.py --train configs/train.yaml --sweep configs/sweep.yaml

Running baseline training alone

In [ ]:
!uv run python -m extractive_qa_mlops.train --config configs/train.yaml


Evaluate best model on SQuAD

In [ ]:
!cd /content/extractive-qa-mlops && uv run python -m extractive_qa_mlops.eval_cli \
  --model_dir "/content/drive/MyDrive/extractive-qa-mlops/artifacts/models/best" \
  --dataset squad --samples 500


MLflow UI via Colab

In [ ]:
!pkill -f "mlflow" || true
!fuser -k 5000/tcp || true
!sleep 1
!ss -ltnp | grep 5000 || echo "5000 not listening"

In [ ]:
!nohup python -m mlflow ui \
  --backend-store-uri file:/content/drive/MyDrive/extractive-qa-mlops/mlruns \
  --host 0.0.0.0 --port 5000 \
  --allowed-hosts "*" \
  --cors-allowed-origins "*" \
  > mlflow.log 2>&1 &

In [ ]:
!ss -ltnp | grep 5000
!curl -I http://127.0.0.1:5000
!tail -n 30 mlflow.log

In [ ]:
from google.colab import output
url = output.eval_js("google.colab.kernel.proxyPort(5000)")
print(url)

MLflow UI via local

In [ ]:
# powershell
uv run python -m mlflow ui --backend-store-uri "file:./mlruns" --port 5000